# Example Usage

## Boilerplate

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np

from pathlib import Path
from scipy import stats

# Import library
sys.path.append(str(Path.cwd().parent.absolute()))
from nems_fingerprint import *
from nems_fingerprint.euler_bernoulli_beam import displacement as ebb_disp

# Plotting
import bokeh
import bokeh.palettes as palettes

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import Column, gridplot

palette = palettes.Category10[10]

output_notebook()

In [31]:
%%html
<!-- So that markdown cells don't stretch across the whole screen -->
<style>
.jp-RenderedMarkdown {
    max-width: 700px;
}
.jp-MarkdownCell {
    max-width: 900px;
}
</style>

In [32]:
# Generic plot settings

line_settings = dict(
    line_width=2,
    muted_alpha=0.1
)

fig_settings = dict(
    width=500,
    height=350
)

tool_settings = dict(
    toolbar_location=None,
    tools=[]
)

## Euler-Bernoulli Beam Simulation

In [33]:
ebb_sim = EBBSimulation(
    boundary_type='clamped-free',
    mode_indices=[1, 2, 3],
    mass_dist=Distribution.constant(1.0),
    position_dist=Distribution.uniform(0.0, 1.0),
    noise_dist=Distribution.normal(0.0, 0.02)
)

learning_events = ebb_sim.sample(n_events=1000)
mass_predictor = MassPredictNN(learning_events)

measurement_phase_shifts = ebb_sim.sample(n_events=2000).freq_shifts
masses = mass_predictor(measurement_phase_shifts)

In [34]:
hist, bin_edges = np.histogram(np.log(masses), density=True, bins=200, range=(-1, 1))
masses_sorted = np.sort(masses)
freq_shifts = learning_events.freq_shifts

settings = dict(margin=20)

p1 = figure(
    x_axis_label='$$\Delta f_1 / f_1$$',
    y_axis_label='$$\Delta f_2 / f_2$$',
    **settings
)
p1.scatter(freq_shifts[:, 0], freq_shifts[:, 1])

p2 = figure(
    x_axis_label='$$\Delta f_1 / f_1$$',
    y_axis_label='$$\Delta f_3 / f_3$$',
    **settings
)
p2.scatter(freq_shifts[:, 0], freq_shifts[:, 2])


p3 = figure(
    x_axis_label='sample # / N',
    y_axis_label='mass',
    y_axis_type='log',
    **settings
)
p3.scatter(np.arange(len(masses)) / len(masses), masses_sorted)


p4 = figure(
    x_axis_label='log(Mass)',
    y_axis_label='pdf',
    **settings
)
p4.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:])



for p in (p1, p2):
    p.xaxis.axis_label_text_font_size = "14pt"
    p.yaxis.axis_label_text_font_size = "14pt"

ps = [p1, p2, p3, p4]
gs = gridplot(ps, ncols=2, width=400, height=400)
show(gs)

## COMSOL: Cylinder
Load and plot eigenmodes from COMSOL export

In [35]:
path = Path('surface_disp.csv')
comsol_modes = extract_eigenmodes(path)
mint = MeshInterp(comsol_modes.pts, comsol_modes.modes)

L = 500
zs = np.linspace(0, 1, 600)
pts = np.array([[0.0, 10, L * z] for z in zs])

values = mint(pts)
ds_norms = np.linalg.norm(values, axis=-1)

maximal_values = np.max(np.vstack((np.max(ds_norms, axis=0), np.max(-ds_norms, axis=0))), axis=0)

ds_values = ds_norms / maximal_values

In [36]:
ps = []

for i in range(comsol_modes.n_modes):
    p = figure(
        title=f'Mode {i + 1}',
        margin=20,
        x_axis_label='z [nm]',
        y_axis_label='mode norm-displacement'
    )
    p.line(zs, ds_values[:, i], line_width=2)
    
    ps.append(p)

for p in ps[1:]:
    p.x_range = ps[0].x_range

    
gs = gridplot(ps, ncols=2, width=300, height=300)
show(gs)

In [37]:
ps = []

ebb_map = (1, 1, 2, 2, 3, 3)


for i in range(comsol_modes.n_modes):
    em = ebb_disp(boundary_type='clamped-clamped', mode=ebb_map[i], x=zs)
    em_normalised = em / np.max(em)
    
    c_mode = values[:, i, 0]
    c_mode_normalised = c_mode / np.max(c_mode)
    
    p = figure(
        title=f'Mode {i + 1}',
        margin=20,
        x_axis_label='z [nm]',
        y_axis_label='x-displacement'
    )
    p.line(zs, c_mode_normalised, line_width=2, legend_label='COMSOL')
    p.line(zs, em_normalised, line_width=2, color=palette[1], legend_label='EBB')
    
    ps.append(p)

for p in ps[1:]:
    p.x_range = ps[0].x_range

    
gs = gridplot(ps, ncols=2, width=300, height=300)
show(gs)

## COMSOL: Mass Adsorption Simulation

In [38]:
path = Path('surface_disp.csv')


# Figure out 3D line distriubtion

def line_sample_generator(n_samples):
    samples = np.zeros((n_samples, 3))
    samples[:, 0] = 10.0
    samples[:, 2] = 500.0 * np.random.rand(n_samples)
    return samples


position_dist = Distribution(line_sample_generator)

M0 = 10e3 # 10kDa
comsol_sim = COMSOLSimulation(
    path,
    mode_indices=[0, 2, 4], # mode indices are zero indexed
    mass_dist=Distribution.constant(M0),
    position_dist=position_dist,
    noise_dist=Distribution.normal(0.0, 1e-6)
)

learning_events = comsol_sim.sample(n_events=1000)
mass_predictor = MassPredictNN(learning_events)

measurement_phase_shifts = comsol_sim.sample(n_events=2000).freq_shifts
masses = mass_predictor(measurement_phase_shifts) / M0

In [39]:
hist, bin_edges = np.histogram(np.log(masses), density=True, bins=200, range=(-1, 1))
masses_sorted = np.sort(masses)
freq_shifts = learning_events.freq_shifts

settings = dict(margin=20)

p1 = figure(
    x_axis_label='$$\Delta f_1 / f_1$$',
    y_axis_label='$$\Delta f_2 / f_2$$',
    **settings
)
p1.scatter(freq_shifts[:, 0], freq_shifts[:, 1])

p2 = figure(
    x_axis_label='$$\Delta f_1 / f_1$$',
    y_axis_label='$$\Delta f_3 / f_3$$',
    **settings
)
p2.scatter(freq_shifts[:, 0], freq_shifts[:, 2])


p3 = figure(
    x_axis_label='sample # / N',
    y_axis_label='mass',
    y_axis_type='log',
    **settings
)
p3.scatter(np.arange(len(masses)) / len(masses), masses_sorted)


p4 = figure(
    x_axis_label='log(Mass)',
    y_axis_label='pdf',
    **settings
)
p4.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:])



for p in (p1, p2):
    p.xaxis.axis_label_text_font_size = "14pt"
    p.yaxis.axis_label_text_font_size = "14pt"

ps = [p1, p2, p3, p4]
gs = gridplot(ps, ncols=2, width=400, height=400)
show(gs)